In [3]:
#1 Setup Mario

In [3]:
#install super mario
!pip install gym_super_mario_bros==7.3.0 nes_py

In [2]:
#downgrade AIGYM to avoid render error
!pip uninstall gym
!pip install gym==0.21.0

^C
^C


In [19]:
#import the game
import gym_super_mario_bros

#import the joypad wrapper
from nes_py.wrappers import JoypadSpace

#import the simplified controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

In [20]:
#Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
env = JoypadSpace(env, SIMPLE_MOVEMENT)

In [3]:
#create a flag
done = True
#loop through each frame in the game
for step in range(10000):
    #start the game to begin with
    if done:
        #start the game
        state = env.reset()
        #do random actions
    state, reward, done, info = env.step(env.action_space.sample())
    #show the game on the screen
    env.render()
#close the game
env.close()

C:\Users\Rhysj\anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


KeyboardInterrupt: 

In [3]:
#2 Preprocess Environment

In [8]:
#Install Pytorch
!pip3 install torch torchvision torchaudio

In [9]:
#Install Stable Baselines
!pip install stable-baselines3[extra]

     -------------------------------------- 177.2/177.2 kB 3.6 MB/s eta 0:00:00
     ---------------------------------------- 35.6/35.6 MB 3.1 MB/s eta 0:00:00
     -------------------------------------- 904.7/904.7 kB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 5.8/5.8 MB 2.8 MB/s eta 0:00:00
     -------------------------------------- 895.9/895.9 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 93.3/93.3 kB 2.7 MB/s eta 0:00:00
     -------------------------------------- 781.3/781.3 kB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 3.6/3.6 MB 3.3 MB/s eta 0:00:00
     -----------------------------

In [21]:
#Import frame stacker wrapper and greyscaling wrapper
from gym.wrappers import GrayScaleObservation

#Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv

#Import Matplotlib -- Did I even use this?
from matplotlib import pyplot as plt

In [22]:
#1 Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')

#2 Simplify the controls
env = JoypadSpace(env, SIMPLE_MOVEMENT)

#3 Greyscale environment
env = GrayScaleObservation(env, keep_dim=True)

#4 Wrap inside dummy environment
env = DummyVecEnv([lambda: env])

#5 Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

In [23]:
#3 Train the RL Model

In [24]:
#Import as for file path management
import os
#import PPO for algorithms
from stable_baselines3 import PPO
#import bse callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [25]:
#Create callback that allows you to save models
class TrainAndLoggingCallback(BaseCallback):
    
    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path
        
    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)
            
    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
            
        return True

In [26]:
#Setup directories
CHECKPOINT_DIR = './train/'
LOG_DIR = '.logs/'

In [27]:
# Setup model saving callback
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [10]:
# Setup the AI model
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001,
           n_steps=512)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [30]:
# Reloading a trained model
model = PPO.load("./train/best_model_70000", env=env)

Wrapping the env in a VecTransposeImage.


In [ ]:
#Train the AI model
model.learn(total_timesteps=1000000, callback=callback)

Logging to .logs/PPO_14
----------------------------
| time/              |     |
|    fps             | 107 |
|    iterations      | 1   |
|    time_elapsed    | 4   |
|    total_timesteps | 512 |
----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 17           |
|    iterations           | 2            |
|    time_elapsed         | 57           |
|    total_timesteps      | 1024         |
| train/                  |              |
|    approx_kl            | 0.0024272157 |
|    clip_fraction        | 0.00586      |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.81        |
|    explained_variance   | 0.249        |
|    learning_rate        | 1e-06        |
|    loss                 | 169          |
|    n_updates            | 1370         |
|    policy_gradient_loss | -0.000133    |
|    value_loss           | 481          |
------------------------------------------

-------------------------------------------
| time/                   |               |
|    fps                  | 9             |
|    iterations           | 13            |
|    time_elapsed         | 728           |
|    total_timesteps      | 6656          |
| train/                  |               |
|    approx_kl            | 7.5560994e-05 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -1.8          |
|    explained_variance   | 0.0158        |
|    learning_rate        | 1e-06         |
|    loss                 | 0.0382        |
|    n_updates            | 1480          |
|    policy_gradient_loss | -0.000299     |
|    value_loss           | 0.113         |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 8             |
|    iterations           | 14            |
|    time_elapsed         | 803 

In [68]:
#4 View the model in action

In [14]:
# Load Model
model = PPO.load('./train/best_model_70000')

In [15]:
#start the game
state = env.reset()
#loop through game
while True:
        action, _ = model.predict(state)
        state, reward, done, info = env.step(action)
        env.render()

C:\Users\Rhysj\anaconda3\lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in ubyte_scalars
  return (self.ram[0x86] - self.ram[0x071c]) % 256


KeyboardInterrupt: 

In [1]:
# Concepts: 
#Foundations of machine learning, Policies
#Reinforcement learning = policy network (CNN and MLP), neural network
#reinforcement learning (Agent, Reward, Environment, Action)
#Proximal Policy Optimization (PPO), 
#Vectorization, Methods and Classes, Wrappers, framestacking, pytorch,
#States, Rewards, AI gym, Stable Baselines, Matplotlib, 
#tensorflow, tensorboard, learning rate (SUPER IMPORTANT), n_steps
#loss going down, variance going up - means model is convergting (tensorborard)